In [72]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

In [78]:
#sl: satisfaction_level —— False: MinMinMaxScaler ; True: StanStandardScaler 
#这里用False和True来表示MinMinMaxScaler还是StanStandardScaler 
#其他同理
#dp:department——False:LabelEncoder; True: OneHotEncoder
#salary也是
#lower_d=False表示不降维 ld_n=1表示降维到1维
def hr_preprocessing(sl = False,le = False,npr=False,amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df = pd.read_csv('./HR（已修改）.csv')
    #1、清洗数据
    df = df.dropna(subset = ['satisfaction_level','last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    #2、得到标注
    label = df['left']
    df = df.drop('left',axis = 1)
    #3、特征选择
    #4、特征处理
        #这里对satisfaction_level等用了归一化 
        #对department和salary用数值化
    scaler_lst = [sl,le,npr,amh,tsc,wa,pl5]#这是一个参数
    column_lst = ['satisfaction_level','last_evaluation', "number_project",
                  "average_monthly_hours", "time_spend_company", "Work_accident",
                  "promotion_last_5years"]
    for i in range(len(scaler_lst)):#遍历scaler_lst
        if not scaler_lst[i]:
            df[column_lst[i]] = \
            MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
            #这里的\只是要换行的意思
        else:
            df[column_lst[i]] = \
            StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
    scaler_lst = [dp,slr]#这是一个参数
    column_lst = ['department','salary']
    for i in range(len(scaler_lst)):#遍历scaler_lst
        if not scaler_lst[i]:#LabelEncoder标签化
            if column_lst[i] == 'salary':
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:#OneHotEncoder独热
            df = pd.get_dummies(df,columns = [column_lst[i]])
            #直接 LabelEncoder()会自动进行升序的处理 所以需要下面的01处理
    #考虑是否降维
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df.values),label
    return df,label#这里也同时返回来label

d = dict([('low',0),('medium',1),('high',2)])#01
def map_salary(s):
    return d.get(s,0)#这里给low medium high定义数字 如果没有找到 则为0




def main():
    print(hr_preprocessing(sl = True,le = True, dp = True, lower_d = False, ld_n = 3))#这里运行为True时 结果会出现负数 因为使用来标准化
    #lower_d = True, ld_n = 3可以降维到3个特征的特征表
if __name__=='__main__':
    main()

(       Unnamed: 0  satisfaction_level  last_evaluation  number_project  \
0               0           -0.936521        -1.087410             0.0   
1               1            0.752756         0.840641             0.6   
2               2           -2.022484         0.957493             1.0   
3               3            0.430989         0.899067             0.6   
4               4           -0.976741        -1.145836             0.0   
5               5           -0.815858        -1.262687             0.0   
6               6           -2.062705         0.314809             0.8   
7               7            1.235407         0.782215             0.6   
8               8            1.114744         1.658602             0.6   
9               9           -0.775637        -1.087410             0.0   
10             10           -0.654974        -1.028984             0.0   
11             11           -2.022484         0.548512             0.8   
12             12            0.913640